In [1]:
import pyiron_workflow as pwf

In [2]:
@pwf.as_inp_dataclass_node
class MyInpData:
    question: str = "still computing"
    answer: int = 42

@pwf.as_out_dataclass_node
class MyOutData:
    question: str = "still computing"
    answer: int = 42

wf = pwf.Workflow("auto_encoding_graph")
wf.to_dataclass = MyInpData("input will be output", 0)
wf.from_dataclass = MyOutData(wf.to_dataclass)
wf.run()

('input will be output', 0)

In [3]:
@pwf.as_macro_node(["q", "a"])
def auto_encoder(q: str, a: int):
    wf = pwf.Workflow("auto_encoding_subgraph")
    wf.to_dataclass = MyInpData(q, a)
    wf.from_dataclass = MyOutData(wf.to_dataclass)
    return wf.from_dataclass.outputs.question, wf.from_dataclass.outputs.answer

macro = auto_encoder()
macro("also when it's a macro", 1)

("also when it's a macro", 1)

It's hidden behind a private variable, but we can investigate the internal state of the subgraph -- i.e. the macro object contains its subgraph's retrospective provenance

In [4]:
macro._wf_macro.to_dataclass.outputs.dataclass.value

MyInpData(question="also when it's a macro", answer=1)

`Workflow` objects are already inspectable, since we can always look at their children:

In [5]:
wf = pwf.Workflow("my_workflow")
wf.to_dataclass = MyInpData("provenance is great", 2)
wf.from_dataclass = MyOutData(wf.to_dataclass.outputs.dataclass)
wf.macro = auto_encoder(wf.from_dataclass.outputs.question, wf.from_dataclass.outputs.answer)
wf.run()

('provenance is great', 2)

In [6]:
wf.macro._wf_macro.to_dataclass.outputs.dataclass.value

MyInpData(question='provenance is great', answer=2)

And we can nest macros, maintaining provenance to arbitrary depth

In [7]:
@pwf.as_macro_node(["deep_q", "deep_a"])
def nested_macro(q: str, a: int):
    wf = pwf.Workflow("nested_macro_subgraph")
    wf.to_dataclass = MyInpData(q, a)
    wf.from_dataclass = MyOutData(wf.to_dataclass)
    wf.sub_macro = auto_encoder(wf.from_dataclass.outputs.question, wf.from_dataclass.outputs.answer)
    return wf.sub_macro.outputs.q, wf.sub_macro.outputs.a

nm = nested_macro()
nm("we might want to make this optional so the garbage collector can save us memory", 3)

('we might want to make this optional so the garbage collector can save us memory',
 3)

In [8]:
nm._wf_macro.sub_macro._wf_macro.to_dataclass.outputs.dataclass.value

MyInpData(question='we might want to make this optional so the garbage collector can save us memory', answer=3)